In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

import pymysql

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup
 
import re
import time

import matplotlib.pyplot as plt
%matplotlib inline 

import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

path = "c:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font', family=font_name)

import seaborn as sns

In [ ]:
options = webdriver.ChromeOptions()

options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-gpu")

options.add_argument("window-size=1920x1080")
options.add_argument("lang=ko_KR")
options.add_argument("user-agent=Chrome/77.0.3865.90")

In [ ]:
# 게임 이름, 가격, 할인 여부, 링크 반환하는 함수
def getPrice(tag):
    getPrice_list=[]
    for i in tqdm_notebook(tag):
        title = i.find('div', {'class': 'tab_item_name'}).text
        link = i['href'].split('com')[1]
        # ori_price: 할인 품목인 경우의 원가
        # fin_price: 할인 품목인 경우의 할인가/할인 품목이 아닌 경우의 원가
        # discount: 할인 여부
        try:
            ori_price = i.find('div', {'class': 'discount_original_price'}).text
            fin_price = i.find('div', {'class': 'discount_final_price'}).text
            discount = 'Yes'
        except:
            try: 
                i.find('div', {'class': 'bundle_base_discount'}).text
                # 세부 링크 접속
                driver.get(i['href'])
                try:
                    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'package_totals_row')))
                    req_link = driver.page_source
                    bs_link = BeautifulSoup(req_link, 'html.parser') 
                    ori_price = bs_link.find('div', {'class':'price bundle_final_package_price'}).text
                    fin_price = bs_link.find('div', {'class':'price bundle_final_price_with_discount'}).text
                    discount = 'Yes'
                except:
                    ori_price, fin_price, discount = 'NA','NA','NA'

            except:
                ori_price = 'NA'
                fin_price = i.find('div', {'class': 'discount_final_price'}).text
                discount = 'No'


        dict_top = {'title': title,
                    'original_price': ori_price,
                    'final_price': fin_price,
                    'discount': discount,
                    'link': link}

        getPrice_list.append(dict_top)
        
    return getPrice_list

## 최고매출(Topseller) 게임 크롤링 및 DB 저장
#### 이름, 가격, 할인 여부, 링크 수집

In [ ]:
conn = pymysql.connect(host = "203.252.196.68",
                       user = 'db1614681', passwd = 'stat1234', db = 'sql1614681',
                       cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()

In [ ]:
myquery = """
CREATE TABLE topsell_price (
    ID BIGINT(7) NOT NULL AUTO_INCREMENT,
    TITLE VARCHAR(100), 
    ORIGINAL_PRICE VARCHAR(100), 
    FINAL_PRICE VARCHAR(100), 
    DISCOUNT VARCHAR(100),
    LINK VARCHAR(100),
    PRIMARY KEY(ID) ) 
    CHARSET=utf8mb4 ;
"""
cur.execute(myquery)

In [ ]:
for i in  tqdm_notebook(range(0,4)):    
    url = "https://store.steampowered.com/games/#p="+str(i)+"&tab=TopSellers"
    driver = webdriver.Chrome(executable_path = 'C:/Users/yumincho/chromedriver/chromedriver.exe',
                          options = options)
    driver.get(url)
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, 'TopSellersTable')))
        req = driver.page_source
    finally:
        req = driver.page_source     
        
    bs = BeautifulSoup(req, 'html.parser')
    tag_15 = bs.find('div', {'id': 'TopSellersRows'}).findAll('a')
    
    # 이름, 가격, 링크 크롤링
    price_list=getPrice(tag_15)
    
    # DB 저장
    query = """
            INSERT INTO topsell_price (TITLE, ORIGINAL_PRICE, FINAL_PRICE, DISCOUNT, LINK)
            VALUES (%s, %s, %s, %s, %s)
            ;
        """
    for i in range(0,15):
        cur.execute( query, (price_list[i]['title'], 
                             price_list[i]['original_price'], 
                             price_list[i]['final_price'], 
                             price_list[i]['discount'],
                             price_list[i]['link']))
    driver.quit()

In [ ]:
conn.commit()
cur.close()
conn.close()